In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt

In [2]:
movies_df = pd.read_csv("C:\\Users\\AFA\\Desktop\\Machine learning files\\movies.csv")
ratings_df = pd.read_csv("C:\\Users\\AFA\\Desktop\\Machine learning files\\ratings_sample.csv")
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
movies_df['year'] = movies_df.title.str.extract(r'\((\d{4})\)', expand=False)
movies_df['title'] = movies_df.title.str.replace(r'\(\d{4}\)', '', regex=True)
movies_df['title'] = movies_df['title'].str.strip()
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [4]:
movies_df = movies_df.drop("genres", axis=1)
movies_df.head()

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995


In [5]:
ratings_df = ratings_df.drop("timestamp", axis=1)
ratings_df.head()

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


In [6]:
userInput = [
            {"title":"Breakfast Club, The", 'rating':5},
            {"title":"Toy Story", 'rating':3.5},
            {"title":"Jumanji", 'rating':2},
            {"title":"Pulp Fiction", 'rating':5},
            {"title":"Akira", 'rating':4.5}
        ]
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


In [7]:
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
inputMovies = pd.merge(inputId, inputMovies)
inputMovies = inputMovies.drop("year", axis=1)

inputMovies

,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,5.0
3,1274,Akira,4.5
4,1968,"Breakfast Club, The",5.0


In [10]:
userSubset = ratings_df[ratings_df["movieId"].isin(inputMovies["movieId"].tolist())]
userSubset

,userId,movieId,rating
19,4,296,4.0
441,12,1968,3.0
479,13,2,2.0
531,13,1274,5.0
681,14,296,2.0
...,...,...,...
3899049,42118,296,3.0
3899165,42121,1968,4.0
3899433,42127,296,4.5
3899764,42128,2,3.0


In [15]:
userSubsetGroup = userSubset.groupby(["userId"])
userSubsetGroup

In [18]:
userSubsetGroup.get_group(1130)

C:\Users\AFA\AppData\Local\Temp\ipykernel_14964\2121309423.py:1: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  userSubsetGroup.get_group(1130)


,userId,movieId,rating
104167,1130,1,0.5
104168,1130,2,4.0
104214,1130,296,4.0
104363,1130,1274,4.5
104443,1130,1968,4.5


In [20]:
userSubsetGroup = sorted(userSubsetGroup, key=lambda x: len(x[1]), reverse=True)

In [21]:
userSubsetGroup[:3]

[((75,),
        userId  movieId  rating
  7507      75        1     5.0
  7508      75        2     3.5
  7540      75      296     5.0
  7633      75     1274     4.5
  7673      75     1968     5.0),
 ((106,),
        userId  movieId  rating
  9083     106        1     2.5
  9084     106        2     3.0
  9115     106      296     3.5
  9198     106     1274     3.0
  9238     106     1968     3.5),
 ((686,),
         userId  movieId  rating
  61336     686        1     4.0
  61337     686        2     3.0
  61377     686      296     4.0
  61478     686     1274     4.0
  61569     686     1968     5.0)]

In [22]:
userSubsetGroup = userSubsetGroup[:100]

In [25]:
#Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
pearsonCorrelationDict = {}

#For every user group in our subset
for name, group in userSubsetGroup:
    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')
    #Get the N for the formula
    nRatings = len(group)
    #Get the review scores for the movies that they both have in common
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = temp_df['rating'].tolist()
    #Let's also put the current user group reviews in a list format
    tempGroupList = group['rating'].tolist()
    #Now let's calculate the pearson correlation between two users, so called, x and y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0

In [27]:
pearsonCorrelationDict

{(75,): 0.8272781516947562,
 (106,): 0.5860090386731182,
 (686,): 0.8320502943378437,
 (815,): 0.5765566601970551,
 (1040,): 0.9434563530497265,
 (1130,): 0.2891574659831201,
 (1502,): 0.8770580193070299,
 (1599,): 0.4385290096535153,
 (1625,): 0.716114874039432,
 (1950,): 0.179028718509858,
 (2065,): 0.4385290096535153,
 (2128,): 0.5860090386731196,
 (2432,): 0.1386750490563073,
 (2791,): 0.8770580193070299,
 (2839,): 0.8204126541423674,
 (2948,): -0.11720180773462392,
 (3025,): 0.45124262819713973,
 (3040,): 0.89514359254929,
 (3186,): 0.6784622064861935,
 (3271,): 0.26989594817970664,
 (3429,): 0.0,
 (3734,): -0.15041420939904673,
 (4099,): 0.05860090386731196,
 (4208,): 0.29417420270727607,
 (4282,): -0.4385290096535115,
 (4292,): 0.6564386345361464,
 (4415,): -0.11183835382312353,
 (4586,): -0.9024852563942795,
 (4725,): -0.08006407690254357,
 (4818,): 0.4885967564883424,
 (5104,): 0.7674257668936507,
 (5165,): -0.4385290096535153,
 (5547,): 0.17200522903844556,
 (6082,): -0.04728

In [28]:
pearson_df = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearson_df.columns = ["similarityIndex"]
pearson_df['userId'] = pearson_df.index
pearson_df.index = range(len(pearson_df))
pearson_df.head()

,similarityIndex,userId
0,0.827278,"(75,)"
1,0.586009,"(106,)"
2,0.832050,"(686,)"
3,0.576557,"(815,)"
4,0.943456,"(1040,)"


In [29]:
topUsers = pearson_df.sort_values(by="similarityIndex", ascending=False)[:50]
topUsers.head()

,similarityIndex,userId
64,0.961678,"(12325,)"
34,0.961538,"(6207,)"
55,0.961538,"(10707,)"
67,0.960769,"(13053,)"
4,0.943456,"(1040,)"


In [34]:
ratings_df['userId'] = ratings_df['userId'].astype(str)
topUserRating = topUsers.merge(ratings_df, left_on="userId", right_on="userId", how="inner")
topUserRating.head()

,similarityIndex,userId,movieId,rating
